# AI SQL
AISQL delivers an easy-to-use composable query language that handles both text and multimodal data through familiar SQL syntax. Our high-performance batch engine processes queries faster through intelligent query optimization, while delivering lower costs than traditional AI solutions. Natively ingest structured and unstructured data into unified multimodal tables, enabling comprehensive insights through familiar SQL analytics for all your data.

## AI SQL Benefits:
### 1) Easy-to-use SQL syntax to build AI pipelines without complex coding
### 2) High-performance processing across all modalities (text, image, audio)
### 3) Lower cost batch processing through optimized architecture to support faster and larger batch jobs.


## Three Major function calls
### AI_FILTER
    AI-powered SQL operator for semantic filtering. You can use the same syntax on a single table (for filtering) or join multiple tables together upon a semantic relationship. 

### AI_AGG/AI_SUMMARIZE_AGG
    Advanced functionalities provide unmatched comprehensive analysis across multiple data entries with a user prompt. This method overcomes contextual window constraints, facilitating extensive multi-record examination with no comparable solutions in the market.

### AI_CLASSIFY
    Functions for easy classification across multi-modal data

## Scenario: Equity Research

Our purpose is to do research on S&P 500 companies. We want to see how they're being talked about in the tech market. We will use internet articles and data from the Snowflake Marketplace.

In this demo we will do the following:

1. Parse document into text from pdf using [PARSE_DOCUMENT](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex)
2. Extract entities using snowflake [structured output](https://docs.snowflake.com/en/user-guide/snowflake-cortex/complete-structured-outputs)
3. AI_FILTER to map companies to S&P 500 tickers ([AI_FILTER (PuPr)](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_filter-snowflake-cortex))
4. Summarize research insights (using [AI_AGG  (PuPr)](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_agg)) across multiple articles


First, download the article PDFs from the repo's `data/EquityDOCS` folder. Then upload to a stage called `@equitydocs`

Next, lets parse research doc into text using [PARSE_DOCUMENT](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex) function from the stage containing the pdfs.

In [ ]:
-- uploaded pdfs into stage @equitydocs
-- (note: please create stage as client side encryption)
use role sysadmin;

CREATE OR REPLACE TABLE raw_docs_text AS SELECT
    relative_path, 
    GET_PRESIGNED_URL(@equitydocs, relative_path) as scoped_file_url, 
TO_VARIANT(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(@equitydocs, relative_path , {'mode': 'layout'})) as raw_text_dict,
    raw_text_dict:content as raw_text
FROM DIRECTORY(@equitydocs);


Lets Review what that parsing did.  We can see that it was able to pull the information from the PDF and put it into a table for us as well as retain some of the information in bullet form and read from the charts as well

In [ ]:
select * from raw_docs_text;

Now we can extract the company and a sentiment from the document using [STRUCTURED OUTPUT](https://docs.snowflake.com/en/user-guide/snowflake-cortex/complete-structured-outputs) reading from the documents we just parsed above.  

In [ ]:
CREATE OR REPLACE TABLE ENTITY_EXTRACTION_EXAMPLE as 
select *,
    AI_COMPLETE(
        model => 'openai-gpt-5-mini',
        prompt => 'You are tasked with extracting companies from a research article. Extract "company" for each company that is identified and the "sentiment", which indicates the sentiment of how the company was referenced.:\n\n' || RAW_TEXT,
        response_format => {
            'type':'json',
            'schema':{
                'type' : 'object',
                'properties': {
                    'company_sentiment': {
                        'type': 'array',
                        'items': {
                            'type': 'object',
                            'properties': {
                                'company': {'type': 'string'},
                                'sentiment': {
                                    'type': 'string',
                                    'description': 'The sentiment with which the company was referenced. May be "positive", "negative", "neutral" or "mixed".',
                                    'enum': ['positive', 'negative', 'mixed', 'neutral']
                                }
                            },
                            'required':['company','sentiment'],
                            'additionalProperties':false
                        }
                    }
                },
                'required':['company_sentiment'],
                'additionalProperties':false
            }
        }
    ) as extraction,
    extraction:company_sentiment::array as list_company_sentiment,
    AI_COMPLETE(
        model => 'openai-gpt-5-mini',
        prompt => PROMPT('Summarize this article into a few sentences {0}', RAW_TEXT)) as summary
    from raw_docs_text
    limit 3
    --QUALIFY ROW_NUMBER() OVER (ORDER BY raw_text) = 3;

-- Extracts companies and their sentiment from article. Takes 1m 6s for one article (gpt5). 
-- For all 7 it takes 6m 2s (gpt5), 56s (gpt5-mini), 46s (gpt5-nano)

Here we can see in the array object returned what the sentiment of each company is.  Looking at the extraction JSON return object we can see the values from the above statement. Double click on a row in the extraction column to see the full values

In [ ]:
select * from  ENTITY_EXTRACTION_EXAMPLE ;

## Access S&P 500 Data from the Snowflake Marketplace
[Similarweb Listing](https://app.snowflake.com/marketplace/listing/GZT1ZA3NHF/similarweb-ltd-s-p-500-by-domain-and-aggregated-by-tickers-sample?search=s%26p+500)

In [ ]:
CREATE OR REPLACE TABLE TICKERS_LIST as select distinct(company_name), ticker
FROM S__P_500_BY_DOMAIN_AND_AGGREGATED_BY_TICKERS_SAMPLE.DATAFEEDS.SP_500
group by 1,2;

First we will need to flatten out this JSON object so we can have a value for each company in its own row.  This will make the joins easier 

In [ ]:
create or replace view flattened_extraction as 
SELECT 
    relative_path as file_name,
    RAW_TEXT,
    summary,
    flattened.value:company::STRING AS Company,
    flattened.value:sentiment::STRING AS Sentiment,
    list_company_sentiment 
FROM 
    entity_extraction_example,
    LATERAL FLATTEN(INPUT => list_company_sentiment) AS flattened;

In [ ]:
select * from flattened_extraction;

# Join company to stock ticker

Now we want to be able to add the information we gathered about each company with a structured dataset. To do that, we need to map each company to a stock ticker.

We'll use [AI_SIMILARITY (PuPr)](https://docs.snowflake.com/en/sql-reference/functions/ai_similarity) and [AI_FILTER (PuPr)](https://docs.snowflake.com/en/sql-reference/functions/ai_filter) to evaluate if the two companies are referring to the same entity.

In [ ]:
create or replace table top_candidates as 
SELECT fe.file_name, 
    fe.raw_text, 
    fe.summary,
    fe.sentiment as sentiment,
    fe.company as extracted_company,
    tl.company_name as mapped_company,
    
    -- Semantic similarity
    AI_SIMILARITY(fe.company, tl.company_name)as sim_score,
    
    ticker as mapped_ticker    
FROM  flattened_extraction fe
CROSS JOIN TICKERS_LIST tl
QUALIFY row_number() OVER (PARTITION BY company, file_name
                         ORDER BY sim_score DESC) <= 3; -- Reducing down to 5 option

In [ ]:
select * from top_candidates;

In [ ]:
-- Entity Disambiguation

create or replace table matched_companies as 
SELECT 
    file_name, 
    raw_text, 
    summary, 
    extracted_company,
    sentiment,
    mapped_company,
    mapped_ticker
FROM top_candidates
where
    AI_FILTER(
        PROMPT('Are all of these three referring to the same company?\nSummary:{0}, Company name:{1}, Another Company Name:{2}', summary, mapped_company, extracted_company));

In [ ]:
Select * from matched_companies;

Lets use another new AISQL function and aggregate insights across multiple documents using [AI_AGG (PuPr)](https://docs.snowflake.com/LIMITEDACCESS/snowflake-cortex/ai_agg)

In [ ]:
SELECT 
    AI_AGG(raw_text, 'What are the key themes from these articles? What changes are happening in the tech industry?')
FROM raw_docs_text;